# 🏠🏠🏠 Projet Kaggle : Estimation DVF 🏠🏠🏠

## Initialisation

### Importation des bibliothèques nécessaires


In [1]:
from eurybia import SmartDrift
import numpy as np
from optbinning import ContinuousOptimalBinning
import os
import pandas as pd
from ydata_profiling import ProfileReport

(CVXPY) Dec 20 11:03:53 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Dec 20 11:03:53 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


INFO:visions.backends:Pandas backend loaded 2.2.2
INFO:visions.backends:Numpy backend loaded 1.26.4
INFO:visions.backends:Pyspark backend NOT loaded
INFO:visions.backends:Python backend loaded


### Lecture des jeux de données d'entrainement et de test


In [2]:
train = pd.read_csv("../data/raw/train.csv", delimiter=",")
test = pd.read_csv("../data/raw/test.csv", delimiter=",")

### Lecture du fichier de description

#### Création d'une fonction pour transformer les modalités


In [3]:
def read_desc_file(filepath: str) -> dict:
    def process_variable_line(line: str) -> tuple[str, str]:
        """Traite une ligne contenant une variable et sa description."""
        variable, description = line.split(":", 1)
        return variable.strip(), description.strip()

    def process_modalite_line(line: str) -> tuple[str, str]:
        """Traite une ligne contenant une modalité et sa description."""
        parts = line.split(None, 1)  # Séparer à la première espace
        modalite = parts[0].strip()  # Premier mot
        desc = parts[1].strip() if len(parts) > 1 else ""  # Reste de la ligne
        return modalite, desc

    def handle_special_cases(
        variable: str, modalite: str, desc: str
    ) -> tuple[str, str]:
        """Gère les cas spécifiques liés à certaines variables."""
        if variable in ["Exterior2nd", "Exterior1st"] and modalite == "Wd":
            modalite = "Wd Sdng"
            desc = "Wood Siding"
        return modalite, desc

    variables = {}
    current_variable = None

    with open(filepath, "r") as file:
        for line in file:
            line = line.strip()

            # Ignorer les lignes vides
            if not line:
                continue

            # Si la ligne contient ":", traiter comme une variable
            if ":" in line:
                variable, description = process_variable_line(line)
                current_variable = variable
                variables[current_variable] = {
                    "description": description,
                    "modalites": {},
                }
            elif current_variable:  # Si on est sur une modalité
                modalite, desc = process_modalite_line(line)
                modalite, desc = handle_special_cases(current_variable, modalite, desc)
                variables[current_variable]["modalites"][modalite] = desc

    return variables

#### Application de la fonction

##### Création du dictionnaire de correspondance


In [4]:
dict_desc = read_desc_file("../data/raw/data_description.txt")

##### Création d'une fonction pour remplacer les modalités, grâce au dictionnaire de correspondance


In [5]:
# Fonction pour remplacer les modalités dans le DataFrame
def replace_mod(df: pd.DataFrame, dict_desc: dict) -> pd.DataFrame:
    for colonne in df.columns:
        if colonne in dict_desc:
            modalites = dict_desc[colonne]["modalites"]
            if modalites:  # Si le dictionnaire des modalités n'est pas vide
                # Convertir les valeurs de la colonne en chaînes de caractères pour correspondre aux clés du dictionnaire
                df[colonne] = (
                    df[colonne].astype(str).str.strip()
                )  # Assurer que c'est en string et enlever les espaces

                # Remplacer les valeurs dans la colonne en utilisant le mappage
                df[colonne] = df[colonne].replace(modalites)

    return df

##### Application de la fonction de modification des modalités sur les jeux de données


In [6]:
# Appliquer la fonction pour remplacer les modalités
train = replace_mod(train, dict_desc)
test = replace_mod(test, dict_desc)

##### Visualisation des transformations sur les premières lignes des jeux de données


In [7]:
train.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,2-STORY 1946 & NEWER,Residential Low Density,65.0,8450,Paved,nan,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",...,0,nan,nan,nan,0,2,2008,Warranty Deed - Conventional,Normal Sale,208500
1,2,1-STORY 1946 & NEWER ALL STYLES,Residential Low Density,80.0,9600,Paved,nan,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",...,0,nan,nan,nan,0,5,2007,Warranty Deed - Conventional,Normal Sale,181500
2,3,2-STORY 1946 & NEWER,Residential Low Density,68.0,11250,Paved,nan,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",...,0,nan,nan,nan,0,9,2008,Warranty Deed - Conventional,Normal Sale,223500


In [8]:
test.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,1-STORY 1946 & NEWER ALL STYLES,Residential High Density,80.0,11622,Paved,nan,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",...,120,0,nan,Minimum Privacy,nan,0,6,2010,Warranty Deed - Conventional,Normal Sale
1,1462,1-STORY 1946 & NEWER ALL STYLES,Residential Low Density,81.0,14267,Paved,nan,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",...,0,0,nan,nan,2nd Garage (if not described in garage section),12500,6,2010,Warranty Deed - Conventional,Normal Sale
2,1463,2-STORY 1946 & NEWER,Residential Low Density,74.0,13830,Paved,nan,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",...,0,0,nan,Minimum Privacy,nan,0,3,2010,Warranty Deed - Conventional,Normal Sale


## Analyse descriptive rapide

### Création d'un rapport html de description

#### Configuration rapide du rapport généré par ydata-profiling.

L'idée, c'est d'avoir un peu de descriptif des données pour avoir une première idée de quoi corriger et comment. Le rapport est simplifié de manière à éviter un fichier trop lourd. Les corrélations seront analysées dans une autre partie.


In [9]:
report = ProfileReport(
    train,
    title="House Pricing Model",
    html={"style": {"theme": "flatly", "full_width": True}},
    correlations=None,
    interactions={"continuous": False},
)

#### Génération du rapport


In [10]:
report.to_file("../outputs/Projet-Kaggle-rapport-descriptif.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Je vous conseil de le consulter [ici](../outputs/Projet-Kaggle-rapport-descriptif.html)

## Évaluation des différences entre train et test

### Eurybia :

Test pour voir si le dataframe train et test ont des différences, que ce soit en typage de données ou simplement en déformation des modalités. Pour faire cette comparaison, je vais utiliser Eurybia, une bibliothèque open-source de la MAIF. Pour plus d'info, regardez [ici](https://eurybia.readthedocs.io/en/latest/index.html). Concrétement, cette bibliothèque sert à tester/comparer des données et d'établir un rapport, suite à une classification binaire supervisée (par défaut, modèle Catboost).

#### Configuration de la nature de la comparaison


In [11]:
SD = SmartDrift(
    df_current=train.drop(columns=["Id", "SalePrice"]),
    df_baseline=test.drop(columns="Id"),
)

#### Compilation et exécution de notre comparaison


In [12]:
%time SD.compile(full_validation=True)

The following variables have mismatching dtypes
             and will not be analyzed: 
 ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']
The variable MSSubClass has mismatching unique values:
[] | ['1-1/2 STORY PUD - ALL AGES']

The variable MSZoning has mismatching unique values:
[] | ['nan']

The variable Utilities has mismatching unique values:
['Electricity and Gas Only'] | ['nan']

The variable Condition2 has mismatching unique values:
["Within 200' of North-South Railroad", 'Adjacent to North-South Railroad', 'Adjacent to East-West Railroad'] | []

The variable HouseStyle has mismatching unique values:
['2.5Fin'] | []

The variable RoofMatl has mismatching unique values:
['Metal', 'Membrane', 'Roll', 'Clay or Tile'] | []

The variable Exterior1st has mismatching unique values:
['Stone', 'Imitation Stucco'] | ['nan']

The variable Exterior2nd has mismatching unique values:
['Other'] | ['nan']

The variable Heatin

Ici, on a deux types d'alertes :

- Les alertes concernant un typage de données différentes
- Les alertes concernant des valeurs uniques de modalités présentent dans un jeu de données et pas dans l'autre.

### Contrôle des différences de typage


In [13]:
list_col = [
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "BsmtFullBath",
    "BsmtHalfBath",
    "GarageCars",
    "GarageArea",
]
for col in list_col:
    print(
        f"Colonne {col} :\n type pour train : {train[col].dtype} | contient des NaN: {train[col].isna().any()}\
        \n type pour test : {test[col].dtype} | contient des NaN: {test[col].isna().any()}"
    )

Colonne BsmtFinSF1 :
 type pour train : int64 | contient des NaN: False        
 type pour test : float64 | contient des NaN: True
Colonne BsmtFinSF2 :
 type pour train : int64 | contient des NaN: False        
 type pour test : float64 | contient des NaN: True
Colonne BsmtUnfSF :
 type pour train : int64 | contient des NaN: False        
 type pour test : float64 | contient des NaN: True
Colonne TotalBsmtSF :
 type pour train : int64 | contient des NaN: False        
 type pour test : float64 | contient des NaN: True
Colonne BsmtFullBath :
 type pour train : int64 | contient des NaN: False        
 type pour test : float64 | contient des NaN: True
Colonne BsmtHalfBath :
 type pour train : int64 | contient des NaN: False        
 type pour test : float64 | contient des NaN: True
Colonne GarageCars :
 type pour train : int64 | contient des NaN: False        
 type pour test : float64 | contient des NaN: True
Colonne GarageArea :
 type pour train : int64 | contient des NaN: False        

Comme pressenti, il s'agit simplement du fait que lorsqu'il y a des NaN, les valeurs entières se transforment en float.

### Bilan sur la différence entre train et test

AUC permettant de voir si les données issues de train et test sont séparé (classification binaire)


In [14]:
SD.plot.generate_indicator(
    fig_value=SD.auc, height=300, width=500, title="AUC train vs test"
)

A priori, train et test sont semblables. Les alertes concernant le typage vont être traitées (imputation des NaN et conversion). Les alertes concernant les modalités vont être traitées au cas par cas. Mais ces deux avertissements semblent relever des cas de figure suffisamment marginaux pour que le modèle de classification n'arrive pas à faire la différence, entre train et test.

## Correction des données

### Valeurs manquantes

- Reprise des valeurs manquantes présentent dans "test", pour les valeurs numériques
- BsmtQual/BsmtExposure/BsmtFinType2 : qualité du sous-sol/exposition du sous-sol/note de la partie sous-sol finie (cas complexe)
  - NaN à convertir en "No Basement" (pas de sous-sol)
- Electrical : circuit électrique (qualité)
  - NaN à convertir en "No Electricity" (pas d'électricité)
- Exterior1st/2nd : matériaux extérieurs
  - NaN à convertir en "Not specified"
- Fence : barrière
  - NaN à convertir en "No Fence" (Pas de Barrière)
- FireplaceQu : qualité des cheminés
  - NaN à convertir en "No Fireplace" (pas de cheminée)
- Functional : fonctionnalité générale de la maison
  - NaN à convertir en "Not specified"
- GarageType/GarageFinish/GarageQual : type de garage/finition du garage/qualité
  - NaN à convertir en "No Garage" (pas de garage)
- KitchenQual : qualité de la cuisine
  - NaN à convertir en "Not specified"
- Lot Frontage : nombre de pieds du terrain connecté avec la route
  - Pour moi, le minimum étant sur le jeu d'entraînement de 21, je décide d'imputer les données à 0 pour le manquant.
  - En clair, si c'est NaN, pour moi, ça veut dire qu'il n'y a aucune partie du terrain longé par une route.
  - S'il y avait eu des zéros déjà présent dans les données, alors j'aurai appliqué un autre traitement.
- MasVnrType : type de placage en maçonnerie
  - NaN à convertir en "no veener" (pas de placage)
- MiscFeature : variable "bazar"
  - NaN à convertir en "No specificity"
- MSZoning : environnement
  - NaN à convertir en "C (all)" (nan sur le jeu de test)
- PoolQC : qualité de la piscine
  - NaN à convertir en "No pool"
- SaleType : type de vente
  - NaN à convertir en "Other"

#### Quantitatif


In [15]:
list_col = [
    "LotFrontage",
    "MasVnrArea",
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "BsmtFullBath",
    "BsmtHalfBath",
    "GarageCars",
    "GarageArea",
]

for col in list_col:
    train[col] = train[col].fillna(0).astype(int)
    test[col] = test[col].fillna(0).astype(int)

#### Qualitatif


In [16]:
train["MasVnrType"] = train["MasVnrType"].replace("nan", "No veener")
test["MasVnrType"] = test["MasVnrType"].replace("nan", "No veener")

for col in test.columns:
    if test[col].dtype == "object":
        if col.startswith("Bsmt"):
            train[col] = train[col].replace("nan", "No basement")
            test[col] = test[col].replace("nan", "No basement")

        elif col.startswith("Garage"):
            train[col] = train[col].replace("nan", "No garage")
            test[col] = test[col].replace("nan", "No garage")

train["Electrical"] = train["Electrical"].replace("nan", "No electricity")
test["Electrical"] = test["Electrical"].replace("nan", "No electricity")

train["FireplaceQu"] = train["FireplaceQu"].replace("nan", "No fireplace")
test["FireplaceQu"] = test["FireplaceQu"].replace("nan", "No fireplace")

train["Fence"] = train["Fence"].replace("nan", "No fence")
test["Fence"] = test["Fence"].replace("nan", "No fence")

train["PoolQC"] = train["PoolQC"].replace("nan", "No pool")
test["PoolQC"] = test["PoolQC"].replace("nan", "No pool")

train["MiscFeature"] = train["MiscFeature"].replace("nan", "No specificity")
test["MiscFeature"] = test["MiscFeature"].replace("nan", "No specificity")

train["MSZoning"] = train["MSZoning"].replace("nan", "C (all)")
test["MSZoning"] = test["MSZoning"].replace("nan", "C (all)")

train["SaleType"] = train["SaleType"].replace("nan", "Other")
test["SaleType"] = test["SaleType"].replace("nan", "Other")

notspec = "Not specified"

for col in ["Exterior1st", "Exterior2nd", "KitchenQual", "Functional"]:
    train[col] = train[col].replace("nan", notspec)
    test[col] = test[col].replace("nan", notspec)

#### Cas spécifique pour la variable MSSubClass : configuration pour la vente

La modalité "1-1/2 STORY PUD - ALL AGES" est présente une seule fois dans le jeu de test et non présente dans train. Donc je choisis de l'imputer à "1-STORY PUD (Planned Unit Development) - 1946 & NEWER". Vu que j'ai l'année de construction, je pourrais regrouper avec les années différentes. Mais finalement, je regarderai lors de la selection de variable si HouseStyle qui ressemble beaucoup est pas mieux.


In [17]:
train["MSSubClass"] = train["MSSubClass"].replace(
    "1-1/2 STORY PUD - ALL AGES",
    "1-STORY PUD (Planned Unit Development) - 1946 & NEWER",
)
test["MSSubClass"] = test["MSSubClass"].replace(
    "1-1/2 STORY PUD - ALL AGES",
    "1-STORY PUD (Planned Unit Development) - 1946 & NEWER",
)

## Traitement spécifique

### Combinaison de variables

#### Street / Alley / PavedDrive : rue / ruelle d'accès au bien / meme chose ou presque (à voir)

- création d'une "conbinaison des deux"
- si Paved pour street ou Alley -> Paved
- si Gravel pour street ou Alley -> Gravel
- si NaN alors "Direct acces"


In [18]:
train["Street"].value_counts(dropna=False)

Street
Paved     1454
Gravel       6
Name: count, dtype: int64

In [19]:
train["Alley"].value_counts(dropna=False)

Alley
nan       1369
Gravel      50
Paved       41
Name: count, dtype: int64

In [20]:
train["PavedDrive"].value_counts(dropna=False)

PavedDrive
Paved               1340
Dirt/Gravel           90
Partial Pavement      30
Name: count, dtype: int64

In [21]:
train[["Id", "Street", "Alley", "PavedDrive"]].groupby(
    ["Street", "Alley", "PavedDrive"], observed=False
).count()

Id
Street Alley  PavedDrive            
Gravel nan    Dirt/Gravel          1
              Paved                5
Paved  Gravel Dirt/Gravel         14
              Partial Pavement     7
              Paved               29
       Paved  Dirt/Gravel          8
              Paved               33
       nan    Dirt/Gravel         67
              Partial Pavement    23
              Paved             1273

Après réflexion, je pense garder uniquement PavedDrive qui est moins déséquilibrée que Street et Alley. Des tests seront tout de même effectués lors de l'étape de modélisation. Je reste vigilant et pense que Street et Alley vont apporter du bruit plutôt qu'autre chose.

### Traitement des dates et transformations en durée

- YearRemodAdd, YearBuilt, YrSold, MoSold, GarageYrBlt
- l'idée c'est d'obtenir la différence entre la date de vente et la date de construction, etc...
- pour l'année de construction de l'extension, on va créer plusieurs variables et regarder celle qui est la meilleure.


#### Différence de date entre l'année de construction et l'année de vente


In [22]:
# Étape 1 : Création de la date de vente à partir de YearSold et MoSold
train["SaleDate"] = pd.to_datetime(
    train["YrSold"].astype(str) + "-" + train["MoSold"].astype(str) + "-01"
)
test["SaleDate"] = pd.to_datetime(
    test["YrSold"].astype(str) + "-" + test["MoSold"].astype(str) + "-01"
)

In [23]:
# Étape 2 : Calculer la différence avec YearBuilt, l'année de construction
# Conversion pour avoir un format de date
train["YearBuilt"] = pd.to_datetime(train["YearBuilt"].astype(str) + "-01-01")
test["YearBuilt"] = pd.to_datetime(test["YearBuilt"].astype(str) + "-01-01")

In [24]:
# Étape 3 : Calculer la différence en années
train["DiffYearsSaleVSBuild"] = (
    train["SaleDate"] - train["YearBuilt"]
).dt.days / 365  # Différence en années
test["DiffYearsSaleVSBuild"] = (
    test["SaleDate"] - test["YearBuilt"]
).dt.days / 365  # Différence en années

#### Même chose entre l'année d'un agrandissement et l'année de construction


In [25]:
# Différence uniquement avec l'année de construction car YearRemodAdd = YearBuilt quand il n'y a pas d'agrandissement
train["YearRemodAdd"] = pd.to_datetime(train["YearRemodAdd"].astype(str) + "-01-01")
test["YearRemodAdd"] = pd.to_datetime(test["YearRemodAdd"].astype(str) + "-01-01")

train["DiffYearsRemodAddVSBuild"] = (
    train["YearRemodAdd"] - train["YearBuilt"]
).dt.days / 365
test["DiffYearsRemodAddVSBuild"] = (
    test["YearRemodAdd"] - test["YearBuilt"]
).dt.days / 365

#### Même chose entre l'année de construction et l'année de construction du garage


In [26]:
train["GarageYrBlt"] = train["GarageYrBlt"].fillna(9999).astype(int)
test["GarageYrBlt"] = test["GarageYrBlt"].fillna(9999).astype(int)

train["GarageYrBlt"] = pd.to_datetime(
    train["GarageYrBlt"].astype(str) + "-01-01", format="%Y-%m-%d", errors="coerce"
)
test["GarageYrBlt"] = pd.to_datetime(
    test["GarageYrBlt"].astype(str) + "-01-01", format="%Y-%m-%d", errors="coerce"
)

In [27]:
train["DiffYearsGarageVSBuild"] = (
    train["GarageYrBlt"] - train["YearBuilt"]
).dt.days / 365
test["DiffYearsGarageVSBuild"] = (test["GarageYrBlt"] - test["YearBuilt"]).dt.days / 365

In [28]:
# Si pas de garage alors on impute a 0
# c'est pour decorreler le fait de ne pas avoir de garage avec d'autres variables qui portent déjà cette information
train["DiffYearsGarageVSBuild"] = train["DiffYearsGarageVSBuild"].fillna(0)
test["DiffYearsGarageVSBuild"] = test["DiffYearsGarageVSBuild"].fillna(0)

### Regroupement de modalités et déséquilibre

Une colonne sera créée à chaque fois, de manière à ne pas perdre d'information. La variable catégorielle qui donnera le plus de performance sera choisie lors de la modélisation.

#### LotShape : forme générale de la propiété

regroupement de "Moderately Irregular" et "Irregular" en "Irregular"


In [29]:
train["LotShape_agg"] = train["LotShape"].replace("Moderately Irregular", "Irregular")
test["LotShape_agg"] = test["LotShape"].replace("Moderately Irregular", "Irregular")

#### LotConfig : configuration du terrain / voisinage

"Frontage on 2 sides of property" et "Frontage on 3 sides of property" regroupé en "Frontage on 2 / 3 sides of property"


In [30]:
frontagesides = "Frontage on 2 / 3 sides of property"

for df in [train, test]:
    df["LotConfig_agg"] = train["LotConfig"].replace(
        {
            "Frontage on 2 sides of property": frontagesides,
            "Frontage on 3 sides of property": frontagesides,
        }
    )

#### Neighborhood : quartier de la ville d'Ames

Regroupement mix entre situations géographiques d'Ames et environnement :

- North Ames (North Ames, Northridge, Northridge Heights, Northwest Ames, Gilbert (ville voisine au nord), Northpark Villa)
- South and west Ames (South & West of Iowa State University, Sawyer, Sawyer West, Somerset)
- Est Ames (Bloomington Heights, Clear Creek, Crawford, Edwards, Iowa DOT and Rail Road)
- Center and university (Old Town, College Creek)
- South Est Ames (Meadow Village, Mitchell)
- High-End Residential Areas (Stone Brook, Timberland, Veenker)
- Green and Natural Areas (Brookside, Bluestem, Briardale)


In [31]:
col = "Neighborhood"

choicelist = [
    "North Ames",
    "South and west Ames",
    "Est Ames",
    "Center and university",
    "South Est Ames",
    "High-End Residential Areas",
    "Green and Natural Areas",
]


for df in [train, test]:
    condlist = [
        df[col].isin(
            [
                "North Ames",
                "Northridge",
                "Northridge Heights",
                "Northwest Ames",
                "Gilbert",
                "Northpark Villa",
            ]
        ),
        df[col].isin(
            [
                "South & West of Iowa State University",
                "Sawyer",
                "Sawyer West",
                "Somerset",
            ]
        ),
        df[col].isin(
            [
                "Bloomington Heights",
                "Clear Creek",
                "Crawford",
                "Edwards",
                "Iowa DOT and Rail Road",
            ]
        ),
        df[col].isin(["Old Town", "College Creek"]),
        df[col].isin(["Meadow Village", "Mitchell"]),
        df[col].isin(["Stone Brook", "Timberland", "Veenker"]),
        df[col].isin(["Brookside", "Bluestem", "Briardale"]),
    ]

    col_name = col + "_agg"

    df[col_name] = np.select(condlist, choicelist, default=notspec)

#### Condition1 : Proximité et situation / élément notables proche du logement

Regroupement proposé :

- Proximité à une voie ferrée (moins de 60 mètres)
- Adjacence à une voie ferrée
- Proximité à une rue principale
- Proximité à des éléments positifs
- Condition normale


In [32]:
col = "Condition1"

choicelist = [
    "Adjacent to a major street",
    "Adjacent to a Railroad",
    "Within 200' of Railroad",
    "Near positive off-site",
    "Normal",
]


for df in [train, test]:
    condlist = [
        df[col].isin(["Adjacent to feeder street", "Adjacent to arterial street"]),
        df[col].isin(
            ["Adjacent to North-South Railroad", "Adjacent to East-West Railroad"]
        ),
        df[col].isin(
            ["Within 200' of East-West Railroad", "Within 200' of North-South Railroad"]
        ),
        df[col].isin(
            [
                "Near positive off-site feature--park, greenbelt, etc.",
                "Adjacent to postive off-site feature",
            ]
        ),
        df[col] == "Normal",
    ]

    col_name = col + "_agg"

    df[col_name] = np.select(condlist, choicelist, default=notspec)

#### OverallQual / OverallCond : Note des materiaux et finitions / conditions du bien

- regroupement de "Very Good", "Excellent" et "Very Excellent"
- regroupement de "Poor", "Very Poor" et "Fair"
- peut être qu'il faudra choisir entre les deux notations qui risques d'être corrélées


In [33]:
cols = ["OverallCond", "OverallQual"]

choicelist = ["Excellent", "Poor"]

for df in [train, test]:
    for col in cols:
        condlist = [
            df[col].isin(["Excellent", "Very Excellent", "Very Good"]),
            df[col].isin(["Poor", "Very Poor", "Fair"]),
        ]

        col_name = col + "_agg"

        df[col_name] = np.select(condlist, choicelist, default=df[col])

#### RoofStyle : type de toiture

J'aurais pu tester WOE Binning ou éventuellement un regroupement avec les toitures rares peut-être, mais je vais attendre avec les valeurs de Shapley lors de la modélisation.

#### Exterior1st/2nd : type d'exterieur

- les regroupements proposés :
- Wood-Based Siding (Revêtements en bois) : Hard Board, Wood Siding (Wd Sdng), Plywood, Wood Shingles
- Brick or Stone Façade (Matériaux de façade en brique ou en pierre) : Brick Face, Brick Common, Stone
- Other Exterior Materials (Autres revêtements) : Stucco, Asbestos Shingles, Asphalt Shingles, Imitation Stucco, Cinder Block


In [34]:
cols = ["Exterior1st", "Exterior2nd"]

choicelist = ["Wood-Based Siding", "Brick or Stone Façade", "Other Exterior Materials"]

for df in [train, test]:
    for col in cols:
        condlist = [
            df[col].isin(["Hard Board", "Wood Siding", "Plywood", "Wood Shingles"]),
            df[col].isin(["Brick Face", "Brick Common", "Stone"]),
            df[col].isin(
                [
                    "Stucco",
                    "Asbestos Shingles",
                    "Asphalt Shingles",
                    "Imitation Stucco",
                    "Cinder Block",
                ]
            ),
        ]

        col_name = col + "_agg"

        df[col_name] = np.select(condlist, choicelist, default=df[col])

#### HeatingQC : qualité de chauffage

- regrouper "fair" et "poor"


In [35]:
train["HeatingQC_agg"] = train["HeatingQC"].replace("Poor", "Fair")
test["HeatingQC_agg"] = test["HeatingQC"].replace("Poor", "Fair")

#### BsmtFullBath/BsmtHalfBath : salle de bain au sous-sol

Cette fois-ci, j'ai envie de tester avec WOE Binning, pour voir comment l'algorithme se comporte et ce qu'il propose.

L'algorithme WOE Binning va essayer de minimiser la variance intra-classe (au sein d'un regroupement de modalités) et maximiser la variance inter-classe (entre les différents regroupements de modalités). La variance ici, c'est la cible (donc le prix des maisons). Ce découpage pourrait être utile surtout pour une régression type GLM par exemple. On laissera la possibilité avec l'arbre booster de voir si le regroupement est plus intéressant que la variable initiale ou pas. Plus d'info [ici](https://gnpalencia.org/optbinning/tutorials/tutorial_continuous.html).

##### Etape 1 :

On choisit x, la variable à discrétiser, et y, la cible continue.


In [36]:
x = train["BsmtFullBath"].values
y = train["SalePrice"].values

##### Etape 2 :

Création de l'objet pour le découpage de notre variable


In [37]:
optbBsmtFullBath = ContinuousOptimalBinning(name="BsmtFullBath", dtype="numerical")

Etape 3 :

Entrainement de l'algorithme sur le jeu d'entraînement "train" et application future sur le jeu "test"


In [38]:
optbBsmtFullBath.fit(x, y)

ContinuousOptimalBinning(name='BsmtFullBath')

##### Etape 4 :

On peut procéder à plusieurs vérifications.

Par exemple, on peut contrôler le statut pour voir si l'algorithme a convergé.


In [39]:
optbBsmtFullBath.status

'OPTIMAL'

Contrôle des découpages proposés


In [40]:
optbBsmtFullBath.splits

array([0.5])

Plusieurs informations sur le découpage et représentation de l'IV (Information Value)


In [41]:
binning_table = optbBsmtFullBath.binning_table

In [42]:
type(binning_table)

optbinning.binning.binning_statistics.ContinuousBinningTable

On peut regarder la table de découpage et constater son lien avec les différents prix des maisons


In [43]:
binning_table.build()

,Bin,Count,Count (%),Sum,Std,Mean,Min,Max,Zeros count,WoE,IV
0,"(-inf, 0.50)",856,0.586301,141686524.0,68098.007851,165521.640187,34900.0,755000.0,0,-15399.555703,9028.780604
1,"[0.50, inf)",604,0.413699,122458422.0,88658.701964,202745.731788,35311.0,745000.0,0,21824.535898,9028.780604
2,Special,0,0.000000,0.0,NaN,0.000000,NaN,NaN,0,-180921.195890,0.000000
3,Missing,0,0.000000,0.0,NaN,0.000000,NaN,NaN,0,-180921.195890,0.000000
Totals,,1460,1.000000,264144946.0,,180921.195890,34900.0,755000.0,0,399066.483382,18057.561208


In [44]:
binning_table.plot()

c:\Users\guill\OneDrive\Bureau\Python\Kaggle\house-prices\house-prices-env\Lib\site-packages\optbinning\binning\binning_statistics.py:1933: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



##### Etape 5 :

Application du découpage en créant une nouvelle colonne et application sur le jeu de test.


In [45]:
train["BsmtFullBath_optb"] = optbBsmtFullBath.transform(x, metric="bins")

x = test["BsmtFullBath"].values
test["BsmtFullBath_optb"] = optbBsmtFullBath.transform(x, metric="bins")

Le même principe va être appliqué pour d'autre variable. Pour eviter la redondance, je vais créer une fonction

#### Création d'une fonction pour plus de simplicité


In [46]:
def bin_feat(
    bin_variable: str,
    bin_target: str,
    df_train: pd.DataFrame,
    df_test: pd.DataFrame,
    transformed_variable_name: str,
    quanti_quali: str,
):
    x = df_train[bin_variable].values

    y = df_train[bin_target].values

    optb_name = "optb_" + bin_variable

    if quanti_quali == "quanti":
        optb_name = ContinuousOptimalBinning(name=bin_variable, dtype="numerical")

    elif quanti_quali == "quali":
        optb_name = ContinuousOptimalBinning(name=bin_variable, dtype="categorical")

    else:
        print('Veuillez renseigner "quanti" ou "quali"')

    optb_name.fit(x, y)

    if optb_name.status != "OPTIMAL":
        print("L'algorithme n'a pas réussi a converger")

    else:
        df_train[transformed_variable_name] = optb_name.transform(x, metric="bins")

        x = df_test[bin_variable].values

        df_test[transformed_variable_name] = optb_name.transform(x, metric="bins")

#### BsmtHalfBath, FullBath et HalfBath


In [47]:
list_col = ["BsmtHalfBath", "FullBath", "HalfBath"]

for col in list_col:
    transformed_variable_name = col + "_optb"
    bin_feat(col, "SalePrice", train, test, transformed_variable_name, "quanti")

#### Functional : Fonctionnalités globales de la maison

- Typical Functionality
- Minor Deductions 1 et 2 et Moderate Deductions -> Minor Deductions
- Major Deductions 1 et 2, Severely Damaged, Salvage only -> Major Deductions


In [48]:
col = "Functional"

choicelist = ["Minor Deductions", "Major Deductions"]

for df in [train, test]:
    condlist = [
        df[col].isin(
            ["Minor Deductions 2", "Minor Deductions 1", "Moderate Deductions"]
        ),
        df[col].isin(
            [
                "Major Deductions 1",
                "Major Deductions 2",
                "Severely Damaged",
                "Salvage only",
            ]
        ),
    ]

    col_name = col + "_agg"

    df[col_name] = np.select(condlist, choicelist, default=df[col])

#### Fireplaces : nombre de cheminés

Regroupement par WOE Binning


In [49]:
bin_feat("Fireplaces", "SalePrice", train, test, "Fireplaces_optb", "quanti")

#### FireplaceQu : qualité des cheminés

- regrouper "fair" et "poor"
- regrouper "Excellent" et "Good"


In [50]:
train["FireplaceQu_agg"] = train["FireplaceQu"].replace(
    "Poor - Ben Franklin Stove", "Fair - Prefabricated Fireplace in basement"
)
test["FireplaceQu_agg"] = test["FireplaceQu"].replace(
    "Poor - Ben Franklin Stove", "Fair - Prefabricated Fireplace in basement"
)

train["FireplaceQu_agg"] = train["FireplaceQu"].replace(
    "Excellent - Exceptional Masonry Fireplace",
    "Good - Masonry Fireplace in main level",
)
test["FireplaceQu_agg"] = test["FireplaceQu"].replace(
    "Excellent - Exceptional Masonry Fireplace",
    "Good - Masonry Fireplace in main level",
)

#### GarageType : type de garage

- A voir. peut etre avec les valeurs de Shapley plus tard
- test avec WOE binning


In [51]:
bin_feat("GarageType", "SalePrice", train, test, "GarageType_optb", "quali")

#### GarageQual

- regrouper "fair" et "poor"
- regrouper "Excellent" et "Good"


In [52]:
train["GarageQual_agg"] = train["GarageQual"].replace("Poor", "Fair")
test["GarageQual_agg"] = test["GarageQual"].replace("Poor", "Fair")

train["GarageQual_agg"] = train["GarageQual"].replace("Excellent", "Good")
test["GarageQual_agg"] = test["GarageQual"].replace("Excellent", "Good")

#### SaleType / SaleCondition : type de vente

- pour l'instant je laisse tel quel.
- A voir si je trouve mieux avec WOE bining ou autre.
- Peut etre avec les valeures de Shapley simplement


In [53]:
bin_feat("SaleType", "SalePrice", train, test, "SaleType_optb", "quali")
bin_feat("SaleCondition", "SalePrice", train, test, "SaleCondition_optb", "quali")

## Remarque de fin sur certaines variables

J'ai un premier avis sur certaines variables. En effet, je pense que quelques-unes d'entre elles n'ont pas ou peu d'intérêt, comportent un déséquilibre trop important, et éventuellement qu'une modalité est dominante. Donc j'ai établi une première liste de variables pour la suite du projet. Je ne souhaite pas supprimer d'information avant plusieurs essais de modélisation. Aucun traitement ne sera donc prévu ici. Seules les dates indiquées seront supprimées.

- Utilities : accès aux énergies
- LowQualFinSF : finition basse qualité (tout étage)
- GarageCars : capacité de voiture du garage (je pense que garage area sera mieux, quitte à la découper en plusieurs catégories)
- 3SsnPorch : véranda spécifique
- ScreenPorch : véranda spécifique
- PoolArea : surface de la piscine
- PoolQC : qualité de la piscine
- MiscFeature / MiscVal : bazard, valeurs inclassables (trop déséquilibré, c'est dommage. Mais peut être uniquement pour les maisons chères)
- Heating : type de chauffage (trop déséquilibré et corrélé avec HeatingQC qui semble mieux)
- Condition2 : proximité et situation/élément notables proche du logement (on ne prendra que la première condition, car condition2 semble hyper déséquilibré)
- RoofMatl : matériaux de la toiture (trop déséquilibré)
- Street / Alley : type de route ou d'allée pour venir à la maison
- MSZoning : contexte du terrain (zone qui l'entoure)
- LandContour : contexte géographique du terrain (en pente, plat, etc.)
- LandSlope : pente de la propriété. Semblable à LandContour
- BldgType : style ou usage structurel principal de la maison vendue
- ExterCond : qualité de la condition extérieure (je pars du principe que c'est corrélé à ExterQual qui est moins déséquilibrée donc plus intéressante)
- BsmtCond : condition du sous-sol (même raisonnement avec BsmtQual)
- BsmtFinType2 : type sous-sol fini (cas complexe)
- BsmtFinSF2 : surface
- CentralAir : à voir, mais c'est une indicatrice hyper déséquilibrée
- Electrical : tester un regroupement peut être, mais je ne sais pas si ça vaut le coup
- Functional : maison déjà habitable
- GarageCond : condition du garage (semblable à qualité GarageQual)
- les dates :
  - YearBuilt
  - MoSold
  - YrSold
  - YearRemodAdd
  - GarageYrBlt

NB : SaleDate, je vais la garder, car elle permettra d'apporter une information supplémentaire. Le marché de l'immobilier étant mouvant en fonction de l'année de vente (crise économique éventuelle), je pourrais avoir une meilleure adaptation (pas de drift de modèle en fonction des années). Elle sera convertie en variable catégorielle.


In [54]:
col_suppr = ["YearBuilt", "MoSold", "YrSold", "YearRemodAdd", "GarageYrBlt"]

for df in [train, test]:
    df.drop(columns=col_suppr, inplace=True)

## Sauvegarde des jeux de données après traitement


In [56]:
# Chemin racine
rootpath = "../data/processed"

# Test de l'existence du répertoire
if not os.path.exists(rootpath):
    # Création du répertoire s'il n'existe pas
    os.makedirs(rootpath)

# Sauvegarde des jeux de données
train.to_csv(
    path_or_buf=os.path.join(rootpath, "train.csv"),
    sep=",",
    encoding="utf-8",
    index=False,
)
test.to_csv(
    path_or_buf=os.path.join(rootpath, "test.csv"),
    sep=",",
    encoding="utf-8",
    index=False,
)